In [18]:
import librosa
import IPython.display as ipd

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from pyAudioAnalysis import ShortTermFeatures

%matplotlib inline

In [7]:
noise_metadata = pd.read_csv("../data/fsd/FSD50K.ground_truth/eval.csv", index_col = 0)
noise_metadata

,labels,mids
fname,,
37199,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
175151,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
253463,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
329838,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
1277,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
...,...,...
266616,Livestock_and_farm_animals_and_working_animals...,"/m/0ch8v,/m/0jbk"
137745,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
418509,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"


In [8]:
available_ids = noise_metadata[~noise_metadata.labels.str.contains("instrument")].index # We want to omit other music
len(available_ids)

8291

In [21]:
def add_noise(x, n_noises, noise_loudness_factor = 0.2, verbose = False):
    x_max_original = x.max()

    for _ in range(n_noises):
        noise_id = random.choice(available_ids)
        noise, _  = librosa.load(f"../data/fsd/FSD50K.eval_audio/{noise_id}.wav", sr = 16000)
        noise = noise * noise_loundess_factor / noise.max() 
        noise_len = len(noise)
        noise_start = random.randint(0, len(x) - noise_len)
        if verbose:
            print(noise_start / fs)
        x[noise_start:noise_start + noise_len] += noise

    x = x * x_max_original / x.max() # Renormalize
    return x

x, fs = librosa.load("../data/clips_45seconds/222.mp3", sr = 16000)
noised = add_noise(x, 15, 0.2, True)
    
ipd.Audio(noised, rate=fs)

4.5570625
8.47175
3.2828125
17.16725
14.750125
15.595625
34.453875
30.0175625
12.581125
34.36075
19.3520625
24.950875
2.491625
3.528125
28.30975


In [20]:
WINDOW = 0.050
STEP = 0.050

import warnings
warnings.filterwarnings('ignore') # Ignore UserWarning when loading every single file

dfs = []

for i in tqdm(range(2, 1001)):
    x, fs = librosa.load(f"../data/clips_45seconds/{i}.mp3", sr = 16000)
    x = add_noise(x, n_noises = 10, noise_loudness_factor = 0.1)
    features, f_names = ShortTermFeatures.feature_extraction(x, fs, WINDOW*fs, STEP*fs)
    df = pd.DataFrame(features.T, columns = f_names)
    df["time"] = [x * STEP for x in range(features.shape[1])]
    df["song_id"] = i
    dfs.append(df)
    df.to_csv(f"../data/augmentations/little_noise/{i}.csv", index = False, float_format='%.6f')

100%|██████████| 999/999 [2:27:32<00:00,  8.86s/it]  


In [22]:
WINDOW = 0.050
STEP = 0.050

import warnings
warnings.filterwarnings('ignore') # Ignore UserWarning when loading every single file

dfs = []

for i in tqdm(range(2, 1001)):
    x, fs = librosa.load(f"../data/clips_45seconds/{i}.mp3", sr = 16000)
    x = add_noise(x, n_noises = 15, noise_loudness_factor = 0.2)
    features, f_names = ShortTermFeatures.feature_extraction(x, fs, WINDOW*fs, STEP*fs)
    df = pd.DataFrame(features.T, columns = f_names)
    df["time"] = [x * STEP for x in range(features.shape[1])]
    df["song_id"] = i
    dfs.append(df)
    df.to_csv(f"../data/augmentations/medium_noise/{i}.csv", index = False, float_format='%.6f')

100%|██████████| 999/999 [2:59:58<00:00, 10.81s/it]  
